# Basic tests.

**IMPORTANT NOTES!**
- **VCCIOB jumper (JP9)** must be in **middle position (1.8V)**, which is different from the AES test notebook;
- **nPOR jumper (JP5)** must be in **bottom position (1-3)**.

In [ ]:
import chipwhisperer as cw

In [ ]:
!ls -l ../fpga/vivado/cw341.runs/impl_1/cw341_top.bit

In [ ]:
bsfile = '../fpga/vivado/cw341.runs/impl_1/cw341_top.bit'
target = cw.target(None, cw.targets.CW340, bsfile=bsfile, force=True)

In [ ]:
target.fpga_buildtime

In [ ]:
from tqdm.notebook import tqdm

def lb_write(addr, data1, data2=None):
    # 1. set lb_action(s) to write:
    action_bits = 0
    if data1 != None:
        action_bits += 2**0
    if data2 != None:
        action_bits += 2**2
    target.fpga_write(0x80, [action_bits])
    
    # 2. write data(s)
    if data1 != None and data2 != None:
        target.fpga_write(0x81, int.to_bytes(data1 + (data2 << 32), length=8, byteorder='little'))
    elif data1 != None:
        target.fpga_write(0x81, int.to_bytes(data1, length=4, byteorder='little'))
    elif data2 != None:
        target.fpga_write(0x85, int.to_bytes(data2, length=4, byteorder='little'))
    else:
        raise ValueError('Must specify some data to write!')
    
    # 3. write address (this triggers the LB write action):
    target.fpga_write(0x82, int.to_bytes(addr, length=4, byteorder='little'))
        

def lb_read(addr, h1en=True, h2en=True):
    # 1. set lb_action(s) to read:
    action_bits = 0
    if h1en:
        action_bits += 2**1
    if h2en:
        action_bits += 2**3
    target.fpga_write(0x80, [action_bits])

    # 2. write address (this triggers the LB read action):
    target.fpga_write(0x82, int.to_bytes(addr, length=4, byteorder='little'))
    
    # 3. read data(s):
    data1 = None
    data2 = None
    if h1en and h1en:
        raw = target.fpga_read(0x81, 8)
        data1 = int.from_bytes(raw[0:4], byteorder='little')
        data2 = int.from_bytes(raw[4:8], byteorder='little')
    elif h1en:
        data1 = int.from_bytes(target.fpga_read(0x81, 4), byteorder='little')
    elif h2en:
        data2 = int.from_bytes(target.fpga_read(0x85, 4), byteorder='little')
    else:
        raise ValueError('Must read from at least one memory!')

    return data1, data2

def set_config(config=0x8fec0000):
    # set config as per do_files:
    lb_write(0x14, config, config) # default = 83 MHz fixed 2x access
    lb_write(0x1c, 0x05, 0x05) # execute the config write

def reset_hyperram(short_pulse=False):
    target.fpga_write(0x86, [1]) # turn off auto test (in case FSM was stuck)
    if short_pulse:
        target.fpga_write(0x84, [0xff])
    else:
        target.fpga_write(0x84, [1])
        target.fpga_write(0x84, [0])

def write_mem_1word(addr, data1, data2=None, h1en=True, h2en=False):
    if h1en:
        addr1=addr
        cmd1 = 0x01
    else:
        addr1=None
        cmd1 = None
    if h2en:
        addr2=addr
        cmd2 = 0x01
    else:
        addr2=None
        cmd2 = None
    
    lb_write(0x10, addr1, addr2) # address
    lb_write(0x14, data1, data2) # data
    lb_write(0x1c, cmd1, cmd2) # write command

def read_mem_1word(addr, h1en=True, h2en=False):
    if h1en:
        addr1=addr
        cmd1=0x04
    else:
        addr1=None
        cmd1=None
    if h2en:
        addr2=addr
        cmd2=0x04
    else:
        addr2=None
        cmd4=None
    
    lb_write(0x10, addr1, addr2) # address
    lb_write(0x1c, cmd1, cmd2) # read command
    return lb_read(0x14, h1en, h2en)

def write_mem_words(addr, data):
    # write 64 bits to each hyperram (total 128 bits)
    # data should be a list of four 32-bit ints
    cmd = 0x3 
    lb_write(0x10, addr, addr) # address
    lb_write(0x14, data[0], data[1])
    lb_write(0x18, data[2], data[3])
    lb_write(0x1c, cmd, cmd) # write command

def read_mem_words(addr, h1en=True, h2en=False):
    # read 64 bits from each hyperram (total 128 bits)
    # returns a list of four 32-bit ints
    cmd = 0x04  
    lb_write(0x10, addr, addr) # address
    lb_write(0x1c, cmd, cmd) # read command
    data = list(lb_read(0x14, True, True))
    data.extend(list(lb_read(0x18, True, True)))
    return data

def auto_hyperram_test(hr1en=True, hr2en=True, start=0, stop=512*1024*1024//32-1):
    target.fpga_write(0x8d, int.to_bytes(start, length=4, byteorder='little'))
    target.fpga_write(0x89, int.to_bytes(stop,  length=4, byteorder='little'))
    target.fpga_write(0x86, [3]) # turn off auto test and clear fail
    config = 16 # set LFSR mode
    if hr1en:
        config += 8
    if hr2en:
        config += 4
    target.fpga_write(0x86, [config]) # turn on test
    wpbar = tqdm(total=stop, desc='writes')
    prevaddr = 0
    addr = 0
    while addr < stop:
        addr = int.from_bytes(target.fpga_read(0x8b, 4), byteorder='little') # current address being read/written
        if addr < prevaddr:
            break
        wpbar.update(addr - prevaddr)
        prevaddr = addr
        time.sleep(0.1)
        busy_stuck = target.fpga_read(0x83, 1)[0] & 6
        if busy_stuck:
            raise ValueError("Hyperram controller appears to be stuck! Re-run the reset_hyperram() and set_config() cells to reset it before trying again.")
    wpbar.close()
    
    rpbar = tqdm(total=stop, desc='reads')
    prevaddr = 0
    addr = 0
    while addr < stop:
        addr = int.from_bytes(target.fpga_read(0x8b, 4), byteorder='little') # current address being read/written
        if addr < prevaddr:
            break
        rpbar.update(addr - prevaddr)
        prevaddr = addr
        time.sleep(0.1)
        busy_stuck = target.fpga_read(0x83, 1)[0] & 6
        if busy_stuck:
            raise ValueError("Hyperram controller appears to be stuck! Re-run the reset_hyperram() and set_config() cells to reset it before trying again.")

    total_errors = int.from_bytes(target.fpga_read(0x87, 4), byteorder='little')
    rpbar.close()
    
    status = target.fpga_read(0x83, 1)[0]

    if status & 16:
        print('Test passed')
    else:
        print('Test FAILED; %d errors' % total_errors)
        if hr1en and hr2en:
            mx = 2
        else:
            mx = 1
        print('Percentage of good reads: %3.2f%%' % ((1-(total_errors/((stop-start)*mx)))*100))
    

def auto_last_address():
    return int.from_bytes(target.fpga_read(0x8b, 4), byteorder='little')

def auto_test_errors():
    return int.from_bytes(target.fpga_read(0x87, 4), byteorder='little')

# Test FPGA register read/writes:

In [ ]:
from tqdm.notebook import tnrange
import random

addr = 0xa # use AES key register
nbytes = 16
iterations = 1000
errors = 0
for i in tnrange(iterations):
    wdata = list(int.to_bytes(random.randrange(2**(8*nbytes)), length=nbytes, byteorder='little'))
    target.fpga_write(addr, wdata)
    rdata = list(target.fpga_read(addr, nbytes))
    if rdata != wdata:
        errors += 1
if errors:
    print('Test FAILED! %d errors' % errors)
else:
    print('Test passed.')

# Test clocks:

Indirectly test the CW340 PLL, via whether the FPGA PLL we use for Hyperram is locked.

In [ ]:
# we only need PLL1 or 2:
target.pll.pll_enable_set(True)
target.pll.pll_outenable_set(False, 0)
target.pll.pll_outenable_set(True, 1)
target.pll.pll_outenable_set(True, 2)

In [ ]:
# PLL clock frequencies must be in range for the FPGA PLL to lock:
target.pll.pll_outfreq_set(40E6, 1)
target.pll.pll_outfreq_set(40E6, 2)

In [ ]:
import time

# PLL1:
target.fpga_write(0, [0])

target.pll.pll_outenable_set(False, 1)
time.sleep(0.1)
assert not target.fpga_read(0x83, 1)[0], 'PLL_CLK1 absent yet FPGA PLL is still locked?'

target.pll.pll_outenable_set(True, 1)
time.sleep(0.1)
assert target.fpga_read(0x83, 1)[0], 'FPGA PLL not locked with PLL_CLK1'

# PLL2:
target.fpga_write(0, [2]) # uses PLL_CLK2_ALT
#target.fpga_write(0, [6]) # uses PLL_CLK2_ORIG

target.pll.pll_outenable_set(False, 2)
time.sleep(0.1)
assert not target.fpga_read(0x83, 1)[0], 'PLL_CLK2 absent yet FPGA PLL is still locked?'

target.pll.pll_outenable_set(True, 2)
time.sleep(0.1)
assert target.fpga_read(0x83, 1)[0], 'FPGA PLL not locked with PLL_CLK2'

# Test LEDs:

In [ ]:
print('The 8 USRLEDs should cycle 5 times, then all flash quickly:')
for i in range(5):
    for j in range(8):
        target.fpga_write(0x0d, [2**j, 1])
        time.sleep(0.05)

target.fpga_write(0x0d, [0, 3])
time.sleep(1)
input("All 8 USLEDs should be flashing now. Press Enter to continue...")
target.fpga_write(0x0d, [0,0])

# Test DIPs:

Make sure VCCIOB jumper (JP9) is set to 1.8V (middle horizontal position).

In [ ]:
import time
input("Set dip switches all to 1, then press enter:")
dips = target.fpga_read(0x0c, 1)[0]
assert dips == 0xff, "{:08b}".format(127)

print("Now set all dip switches to 0:")

lastdips = dips
while True:
    dips = target.fpga_read(0x0c, 1)[0]

    change = dips ^ lastdips

    if change:
        for i in range(0, 8):
            if change == 1<<i:
                print("SW%d: %d" % (i, ((dips & 2**i)>>i)))

    lastdips = dips

    time.sleep(0.1)

    if dips == 0:
        print("All dip switches 0, test passed.")
        break        


# Test SRAM:

In [ ]:
import time

# set wait times:
target.fpga_write(0x44, [6])
target.fpga_write(0x45, [4])

target.fpga_write(0x40, [1]) # enable SRAM test
time.sleep(0.5)

srampass = target.fpga_read(0x41, 1)[0]
target.fpga_write(0x40, [0]) # disable SRAM test

if srampass:
    runs = int.from_bytes(list(target.fpga_read(0x43, 4)), byteorder='little')
    print('SRAM test passed. Wrote and read full memory %d times.' % runs)
else:
    print('*** SRAM test failed! ***')


# Test Hyperrams:

The hyperram controller is quite sensitive to the clock frequency. If the controller gets stuck, try a slightly different (higher or lower) frequency, e.g. 35 MHz or 45 MHz.

In [ ]:
HR_CLOCK_RATE = 40e6
target.pll.pll_outfreq_set(HR_CLOCK_RATE, 2)

In [ ]:
reset_hyperram()

In [ ]:
#set_config(0x8f140a07) # 166 MHz variable
#set_config(0x8f1c0a07) # 166 MHz fixed
#set_config(0x8f0c0806) # 133 MHz fixed
#set_config(0x8ffc0605) # 100 MHz fixed
set_config(0x8fec0404) # 83 MHz fixed

Automated check over the full memory space:

In [ ]:
# Test memory 1: should have no errors
auto_hyperram_test(True, False)
#auto_hyperram_test(True, False, stop=261000)

In [ ]:
# If stuck, report last read or write address, and number of errors:
print('Last read or write address (if stuck): %d' % auto_last_address())
print('Numer of errors: %d' % auto_test_errors())

In [ ]:
# Test memory 2: expect a small number of errors
auto_hyperram_test(False, True)

In [ ]:
# If stuck, report last read or write address, and number of errors:
print('Last read or write address (if stuck): %d' % auto_last_address())
print('Numer of errors: %d' % auto_test_errors())

Python-driven check -- much slower; can be useful to diagnose issues.

Unlike the automated test this does not validate the full memory (it would take a few days with this approach!), however it does fully exercise the interface of both memories (with the default settings used below).

In [ ]:
WORDS = 32
# CHECKS controls which of the four words per burst are verified:
CHECKS = [0,1,2,3] # check burst read/write over both memories
#CHECKS = [0,2] # check burst read/write for memory 1 only
#CHECKS = [1,3] # check burst read/write for memory 2 only
#CHECKS = [0] # check only the first 32 bits for memory 1 only
#CHECKS = [1] # check only the first 32 bits for memory 2 only

# start address for the test (divide what auto_last_address() reports by 2):
START = 0
#START = 233000//2

In [ ]:
import random
from tqdm.notebook import tnrange

target.fpga_write(0x86, [3]) # turn off auto test and clear fail

wdata = []
errors = 0
goods = 0
for i in tnrange(START, START+WORDS, desc='Writing'):
    data = []
    for j in range(4):
        data.append(random.randint(0, 2**32-1))
        #print('i=%3d, j=%3d: writing %08x' % (i, j, data[-1]))
    write_mem_words(i*2, data)
    wdata.append(data)
    busy_stuck = target.fpga_read(0x83, 1)[0] & 6
    if busy_stuck:
        raise ValueError("Hyperram controller appears to be stuck on address %08x! Re-run the reset_hyperram() and set_config() cells to reset it before trying again." % (i*2))


for i in tnrange(START, START+WORDS, desc='Reading'):
    rdata = read_mem_words(i*2)
    busy_stuck = target.fpga_read(0x83, 1)[0] & 6
    if busy_stuck:
        raise ValueError("Hyperram controller appears to be stuck on address %08x! Re-run the reset_hyperram() and set_config() cells to reset it before trying again." % (i*2))
    rdata_filtered = []
    wdata_filtered = []
    for j in CHECKS:
        rdata_filtered.append(rdata[j])
        wdata_filtered.append(wdata[i-START][j])                     
    if rdata_filtered != wdata_filtered:
        for j in range(len(CHECKS)):
            expect = wdata_filtered[j]
            got = rdata_filtered[j]
            if got != expect:
                errors += 1
                if errors < 10:
                    print('i=%3d, j=%3d: got %08x, expected %08x, diff %08x, bits wrong: %3d' % (i, j, got, expect, got ^ expect, bin(got ^ expect).count('1')))
            else:
                goods += 1
    else:
        if goods == 0:
            print('Good read for i=%3d' % i)
        goods += len(CHECKS)

print('Percentage of good reads: %d%%' % (goods/(errors+goods)*100))
assert errors == 0, '%d errors!' % errors

# Check XADC:

In [ ]:
assert target.xadc_status == 'good'
assert target.temp < 60 # somewhat arbitrary